# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
%load_ext dotenv
%dotenv

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

price_data_path = os.getenv("PRICE_DATA")
stock_files = glob(os.path.join(price_data_path, "*.csv"))

dt_list = []
for file in stock_files:
    dt = dd.read_csv(file).assign(
        Ticker=os.path.basename(file).replace(".csv", "")
    )
    dt_list.append(dt)

stock_prices = dd.concat(dt_list, axis=0, ignore_index=True)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
dd_shift = stock_prices.groupby("Ticker", group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1),
        Adj_Close_lag_1 = x['Adj Close'].shift(1)
    )
)

C:\Users\jairo\AppData\Local\Temp\ipykernel_12732\3930762097.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = stock_prices.groupby("Ticker", group_keys=False).apply(
c:\Users\jairo\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\groupby.py:210: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)


In [5]:
dd_returns = dd_shift.assign(
    Returns = lambda x: x['Close'] / x['Close_lag_1'] - 1
)

In [6]:
dd_high_low = dd_returns.assign(
    Hi_low_range = lambda x: x['High'] - x['Low']
)

In [7]:
dd_feat = dd_high_low

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [8]:
dd_feat = dd_feat.compute()

c:\Users\jairo\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\groupby.py:210: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)
c:\Users\jairo\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\groupby.py:210: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)
c:\Users\jairo\miniconda3\envs\dsi_participant\lib\site-pack

In [9]:
dd_feat['Rolling_mean_10'] = dd_feat.groupby('Ticker')['Returns'].transform(lambda x: x.rolling(10).mean())

In [10]:
dd_feat.head(15)

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Close_lag_1,Adj_Close_lag_1,Returns,Hi_low_range,Rolling_mean_10
0,2019-09-19,18.750000,20.139999,17.610001,20.110001,20.110001,11486300.0,PING,NaN,NaN,NaN,2.529999,NaN
1,2019-09-20,20.180000,21.000000,19.150000,19.309999,19.309999,2011500.0,PING,20.110001,20.110001,-0.039781,1.850000,NaN
2,2019-09-23,19.400000,19.950001,18.540001,18.809999,18.809999,668100.0,PING,19.309999,19.309999,-0.025893,1.410000,NaN
3,2019-09-24,18.889999,19.350000,16.510000,18.510000,18.510000,1931200.0,PING,18.809999,18.809999,-0.015949,2.840000,NaN
4,2019-09-25,18.469999,19.070000,17.680000,18.000000,18.000000,1253900.0,PING,18.510000,18.510000,-0.027553,1.389999,NaN
5,2019-09-26,17.980000,18.160000,17.000000,17.040001,17.040001,881600.0,PING,18.000000,18.000000,-0.053333,1.160000,NaN
6,2019-09-27,17.280001,17.500000,16.510000,16.520000,16.520000,1100600.0,PING,17.040001,17.040001,-0.030516,0.990000,NaN
7,2019-09-30,16.520000,17.350000,16.059999,17.250000,17.250000,634400.0,PING,16.520000,16.520000,0.044189,1.290001,NaN
8,2019-10-01,17.209999,17.600000,16.139999,16.430000,16.430000,891900.0,PING,17.250000,17.250000,-0.047536,1.460001,NaN
9,2019-10-02,16.010000,16.165001,15.510000,15.970000,15.970000,704200.0,PING,16.430000,16.430000,-0.027998,0.655001,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

**Comments**
- Strictly, no. I ran the same logic using Pandas and got the same results. However, with Dask you don't get the figures for the rolling average unless you transition to a Pandas dataframe first
- For real-world-size datasets, it is better to use Dask because it provides better performance through *lazy evaluation*

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.